In [1]:
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px
%pylab inline

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 40000)
pd.set_option('display.max_rows', None)

df_safegraph_poi = pd.read_csv(os.path.join('..', 'dewey_data_filtered','SafeGraph_POI_Yellowstone_200miRadius.csv.gz'))
df_safegraph_spend = pd.read_csv(os.path.join('..', 'dewey_data_filtered','SafeGraph_SPEND_Yellowstone_200miRadius.csv.gz'))

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
def map_points(fig,df):

    gdf_safegraph_poi = gpd.GeoDataFrame(df,
                                         geometry=gpd.points_from_xy(df.LONGITUDE, df.LATITUDE))
    fig = px.scatter_mapbox(
        gdf_safegraph_poi,
        lat='LATITUDE',
        lon='LONGITUDE',
        hover_name="LOCATION_NAME",
        hover_data=["CITY"],
        zoom=6,
        opacity=0.5)
  
    return fig

In [4]:
accomodation_naics = '721'
mask = [ str(ncode)[:3]==accomodation_naics for ncode in list(df_safegraph_poi['NAICS_CODE']) ]
df_hotels = df_safegraph_poi[mask]

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_filtered = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_hotels['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

print('(after filter)  # of rows: ', len(df_filtered))
print(df_filtered.head())

(after filter)  # of rows:  12523
               PLACEKEY                        SAFEGRAPH_BRAND_IDS        BRANDS   SPEND_DATE_RANGE_START     SPEND_DATE_RANGE_END  RAW_TOTAL_SPEND  RAW_NUM_TRANSACTIONS  RAW_NUM_CUSTOMERS  MEDIAN_SPEND_PER_TRANSACTION  MEDIAN_SPEND_PER_CUSTOMER SPEND_PER_TRANSACTION_PERCENTILES                                       SPEND_BY_DAY                       SPEND_PER_TRANSACTION_BY_DAY                               SPEND_BY_DAY_OF_WEEK                                         DAY_COUNTS  SPEND_PCT_CHANGE_VS_PREV_MONTH  SPEND_PCT_CHANGE_VS_PREV_YEAR  ONLINE_TRANSACTIONS  ONLINE_SPEND          TRANSACTION_INTERMEDIARY          SPEND_BY_TRANSACTION_INTERMEDIARY                 BUCKETED_CUSTOMER_FREQUENCY               MEAN_SPEND_PER_CUSTOMER_BY_FREQUENCY                     BUCKETED_CUSTOMER_INCOMES                  MEAN_SPEND_PER_CUSTOMER_BY_INCOME                                 CUSTOMER_HOME_CITY         RELATED_CROSS_SHOPPING_PHYSICAL_BRANDS_PCT        RELATE

In [5]:
fig = figure()
fig = map_points(fig,df_hotels)
fig.update_layout(title="Accomadations in Greater Yellowstone Region",height=800,width=1200,mapbox_style="open-street-map")

<Figure size 640x480 with 0 Axes>

In [11]:
import ast

datetime_start = pd.to_datetime('2022-01-01')
datetime_end = pd.to_datetime('2022-12-31')

df = df_filtered.copy()

df_filtered['SPEND_DATE_RANGE_START'] = pd.to_datetime(df_filtered['SPEND_DATE_RANGE_START'])
df_filtered['SPEND_DATE_RANGE_END'] = pd.to_datetime(df_filtered['SPEND_DATE_RANGE_END'])

df_filtered_2022 = df[(df_filtered['SPEND_DATE_RANGE_START'] >= datetime_start) &
                                 (df_filtered['SPEND_DATE_RANGE_END'] <= datetime_end)]

df = df_filtered_2022.copy()

# Convert string representations of lists in 'SPEND_BY_DAY' to actual lists
df['SPEND_BY_DAY'] = df['SPEND_BY_DAY'].apply(ast.literal_eval)

# Add a new column called 'DATE_OF_MONTH', as a list of integers from 1 to the end of the month (length of the 'SPEND_BY_DAY' list)
df['DATE_OF_MONTH'] = df['SPEND_BY_DAY'].apply(lambda x: list(range(1, len(x) + 1)))

df_spend_by_day = df.explode(['SPEND_BY_DAY', 'DATE_OF_MONTH'])

df_spend_by_day['SPEND_BY_DAY'] = df_spend_by_day['SPEND_BY_DAY'].astype(float)
df_spend_by_day['DATE_OF_MONTH'] = df_spend_by_day['DATE_OF_MONTH'].astype(int)

df_spend_by_day['DATE'] = pd.to_datetime(df_spend_by_day['SPEND_DATE_RANGE_START']) + pd.to_timedelta(df_spend_by_day['DATE_OF_MONTH'] - 1, unit='D')
#df_spend_by_day['DATE'] = df_spend_by_day['DATE'].dt.strftime('%m-%d')

grouped_22_lodging = df_spend_by_day.groupby('DATE')['SPEND_BY_DAY'].sum()

C:\Users\tjbla\AppData\Local\Temp\ipykernel_7412\2846758666.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_7412\2846758666.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
px.line(grouped_22_lodging)